In [2]:
import pandas as pd
from meteostat import Point, Hourly
from datetime import datetime, timedelta

In [3]:
df_coord = pd.read_csv('dados/distritos_lat_lon.csv')

In [8]:
df_coord

,Unnamed: 0,Distrito,Latitude,Longitude
0,0,Água Rasa,-23.565372,-46.573697
1,1,Alto de Pinheiros,-23.549906,-46.707642
2,2,Anhanguera,-23.432908,-46.788534
3,3,Aricanduva,-23.578024,-46.511454
4,4,Artur Alvim,-23.540469,-46.489791
...,...,...,...,...
91,91,Vila Medeiros,-23.487707,-46.584496
92,92,Vila Prudente,-23.592335,-46.574961
93,93,Vila Sônia,-23.598656,-46.739294
94,94,São Domingos,-23.493876,-46.742194


In [6]:
start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
end_date = start_date + timedelta(days=5)

all_data = []

for index, row in df_coord.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    location = Point(latitude, longitude)
    
    # Obter dados diários do Meteostat
    data = Hourly(location, start_date, end_date).fetch()
    
    # Adicionar uma coluna com o nome do distrito
    data['Distrito'] = row['Distrito']
    
    # Adicionar os dados à lista
    all_data.append(data)

final_data = pd.concat(all_data)
    
# Adicionando colunas para ano, mês e dia para partições
final_data['Ano'] = final_data.index.year
final_data['Mes'] = final_data.index.month

final_data.reset_index(inplace=True)

In [7]:
final_data

,index,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,Distrito,Ano,Mes
0,2024-09-09 00:00:00,24.2,11.5,45.0,0.0,NaN,136.0,9.3,NaN,1015.6,NaN,1.0,Água Rasa,2024,9
1,2024-09-09 01:00:00,23.8,11.2,45.0,0.0,NaN,126.0,7.4,NaN,1015.6,NaN,1.0,Água Rasa,2024,9
2,2024-09-09 02:00:00,23.4,11.4,47.0,0.0,NaN,114.0,7.4,NaN,1015.6,NaN,1.0,Água Rasa,2024,9
3,2024-09-09 03:00:00,23.1,11.2,47.0,0.0,NaN,101.0,7.4,NaN,1015.6,NaN,1.0,Água Rasa,2024,9
4,2024-09-09 04:00:00,22.2,10.0,46.0,0.0,NaN,89.0,7.4,NaN,1015.1,NaN,1.0,Água Rasa,2024,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11369,2024-09-13 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lajeado,2024,9
11370,2024-09-13 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lajeado,2024,9
11371,2024-09-13 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lajeado,2024,9
11372,2024-09-13 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lajeado,2024,9


In [14]:
import requests
import pandas as pd

lista = []

for index, row in df_coord.iterrows():
    # URL da API
    URL = 'https://api.openweathermap.org/data/2.5/forecast'
    latitude = row['Latitude']
    longitude = row['Longitude']

    # Parâmetros da solicitação
    params = {
        'lat': latitude,
        'lon': longitude,
        'appid': 'eb27e58eb68d175624e79e4efed521eb',
        'units': 'metric',  # Use 'imperial' para Fahrenheit
        'lang': 'pt'  # Idioma dos detalhes (opcional)
    }

    # Fazer a solicitação para a API
    response = requests.get(URL, params=params)
    data = response.json()

    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Extrair os dados necessários
        for item in data['list']:
            detalhes = {
                'Data e Hora': item['dt_txt'],
                'Temperatura': item['main']['temp'],
                'Temperatura Mínima': item['main']['temp_min'],
                'Temperatura Máxima': item['main']['temp_max'],
                'Umidade': item['main']['humidity'],
                'Pressão': item['main']['pressure'],
                'Condições': item['weather'][0]['description'],
                'Velocidade do Vento': item['wind']['speed'],
                'Nuvens': item['clouds']['all'],
                'Distrito': row['Distrito']
            }
            lista.append(detalhes)

    else:
        print(f"Erro ao obter dados: {data.get('message', 'Desconhecido')}")

# Criar um DataFrame com os dados
df = pd.DataFrame(lista)

In [15]:
df

,Data e Hora,Temperatura,Temperatura Mínima,Temperatura Máxima,Umidade,Pressão,Condições,Velocidade do Vento,Nuvens,Distrito
0,2024-09-09 00:00:00,26.55,26.55,27.12,54,1020,céu limpo,3.35,0,Água Rasa
1,2024-09-09 03:00:00,26.09,25.17,26.09,46,1020,céu limpo,3.51,1,Água Rasa
2,2024-09-09 06:00:00,24.44,23.39,24.44,42,1019,céu limpo,3.30,1,Água Rasa
3,2024-09-09 09:00:00,21.69,21.69,21.69,40,1019,céu limpo,2.82,1,Água Rasa
4,2024-09-09 12:00:00,25.85,25.85,25.85,26,1020,céu limpo,3.06,1,Água Rasa
...,...,...,...,...,...,...,...,...,...,...
3835,2024-09-13 09:00:00,20.30,20.30,20.30,33,1014,céu pouco nublado,2.19,20,Lajeado
3836,2024-09-13 12:00:00,28.16,28.16,28.16,22,1014,nuvens dispersas,3.04,33,Lajeado
3837,2024-09-13 15:00:00,34.74,34.74,34.74,13,1012,céu limpo,2.06,0,Lajeado
3838,2024-09-13 18:00:00,33.72,33.72,33.72,20,1009,céu limpo,4.37,0,Lajeado
